In [1]:
import requests as rqs
from bs4 import BeautifulSoup as bs
import re,json # re 是正規表達式

my_headers = {
    'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.71 Safari/537.36'
}

listData = []

url = 'https://www.zeczec.com/'

pages = 1 # 可以改 ,你想去哪一頁就去哪一頁

def getMainData():
    for page in range(1, pages + 1):
        r = rqs.get(url = url + f"categories?page={page}", headers=my_headers)
#         print(r.text)
        soup = bs(r.text,"lxml")
#         print(soup)
        for a in soup.select('div.flex.gutter3-l a.db'): #取得所有圖片超連結
#             print(a.select_one('div.aspect-ratio-project-cover')) 為了看到圖片的屬性 data-bg-src
            strStyle = a.select_one('div.aspect-ratio-project-cover')['data-bg-src']  #有圖片屬性就可以取得圖片
#             print(strStyle)
            regexImg = r"https:\/\/s3-ap-northeast-1\.amazonaws\.com\/zeczec-prod\/asset_\d+_image_big\.(jpe?g|png)"  # 圖片網址的正規表達式
            matchImg = re.match(regexImg,strStyle) # (圖片的正規表達式的變數名稱,要測試的字串)
            strImg = matchImg[0] #結果放在這裡面
#             print(strImg) # 取得其中圖片的資料
            strLink = url+a['href'] #取得內文連結
#             print(strLink) 
            strTitle = a.select_one('h3.b').get_text() # 取得每一個 Title
#             print(strTitle)
            listData.append({
                "cover":strImg,
                "link":strLink,
                "title":strTitle
            }) 
#     print(listData) # 取得每一筆 封面 , 連結 , 標題 用 list 裝起來
def getDetailData():# 取得更詳細的網頁資訊 
    for index,_dict in enumerate(listData): # 用列舉 index 0-11 , _dict會對應到上面的listData裡面的字典資訊
        r = rqs.get(url = _dict['link'],headers = my_headers)
#         print(r.text)
        soup = bs(r.text,"lxml")
        strPrice = soup.select_one('div.js-sum-raised').get_text()
        regexPrice = r"\d+"
        strPrice = ''.join(re.findall(regexPrice, strPrice)) # ''.join 把 list 合併再一起,單純只輸出數字
#         print(strPrice)
        strBacker = soup.select_one('span.js-backers-count').get_text() # 取得贊助人數
#         print(strBacker)
        strTime = "longterm" 
        if soup.select_one('span.js-time-left') != None:
            strTime = soup.select_one('span.js-time-left').get_text() #會印出43天
            regexTime = r"\d+"
            strTime = re.search(regexTime,strTime)[0] # 沒有天
#         print(strTime) # 印出剩餘天數
        dictDuration = {'begin':'' , 'end':''}
        strDuration = soup.select_one('div.mb2.f7').get_text()
        regexDuration = r"(\d{4}\/\d{2}/\d{2}\s\d{2}:\d{2})(\s–\s(\d{4}\/\d{2}\/\d{2}\s\d{2}:\d{2}))?"
        matchDuration = re.search(regexDuration,strDuration)
        dictDuration['begin'] = matchDuration[1] # 第一組數字依定會存在 begin 起始時間
        if matchDuration[3] != None: 
            dictDuration['end'] = matchDuration[3]
#         print(dictDuration)
        listData[index]['price'] = strPrice
        listData[index]['backer'] = strBacker
        listData[index]['time'] = strTime
        listData[index]['Duration'] = strDuration
        listData[index]['comments'] = _dict['link'] + "/comments"
        listData[index]['faqs'] = _dict['link'] + "/faqs"
#     print(listData) 印出所有我取得的內容
def saveJson(): # 變成 json 檔 , 比較好閱讀
    with open("crowdfunding.json","w", encoding = "utf-8") as file:
        file.write( json.dumps(listData, ensure_ascii=False, indent=4))
if __name__ == "__main__":
    getMainData()
    getDetailData()
    saveJson()